# A EfficientNet-B2 Based Model Coding in Pytorch Lightning

## 🧠 Model Overview

This model was developed for the CSIRO competition using PyTorch Lightning.
It leverages a dual-branch architecture designed to process paired image halves and learn their combined representations efficiently.

## ⚙️ Data Preprocessing

Each input image is split in half along the center, and both halves are fed into the model simultaneously.
This setup allows the network to capture cross-regional relationships within each sample.

## 🏗️ Model Architecture

The model structure follows this pipeline:

2 × EfficientNet-B2 (shared weights) -> Linear -> ReLU -> Linear

Only the last block of the EfficientNet backbone is trainable, keeping the remaining layers frozen for stability and efficiency.

## ⚠️ Important Notice

Version 2 and Version 1 of the model have different architectures and are not compatible.
Please make sure to use the matching training notebook for the correct version.

## Code

In [ ]:
# I would like to thank these generous sharings which make this notebook possible:
# https://www.kaggle.com/code/easterndundrey/csiro-gold-solution
# https://www.kaggle.com/code/easterndundrey/bad-assumptions?scriptVersionId=272451747
# https://www.kaggle.com/code/mks2192/csiro-notebook-training
# https://www.kaggle.com/code/mks2192/csiro-image2biomass

import timm
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torchvision.transforms as T
from torchvision.transforms import functional as TF
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
import random
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold

In [ ]:
# ======== Load Datafile ========
test_df = pd.read_csv('/kaggle/input/csiro-biomass/test.csv')
test_df["id"] = test_df["sample_id"].str.split("__").str[0]
image_paths_df = test_df[["id","image_path"]].drop_duplicates("image_path").reset_index(drop=True)

In [ ]:
# ======== Seed ========
def seed_everything(seed: int = 114514):
    pl.seed_everything(seed, workers=True)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
seed_everything(114514)

In [ ]:
# ======== Dataset ========
class InferenceDataset(Dataset):
    def __init__(self, df, full_image_aug=None, per_half_transform=None):
        self.df = df
        self.full_image_aug = full_image_aug
        self.per_half_transform = per_half_transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image = Image.open('/kaggle/input/csiro-biomass/' + row["image_path"]).convert("RGB")
    
        base = self.full_image_aug(image) if self.full_image_aug is not None else image
    
        left = base.crop((0, 0, 1000, 1000))
        right = base.crop((1000, 0, 2000, 1000))
    
        if self.per_half_transform is not None:
            left = self.per_half_transform(left)
            right = self.per_half_transform(right)
        else:
            left = F.to_tensor(left)
            right = F.to_tensor(right)
    
        imgs = torch.stack([left, right], dim=0)   # (2, C, H, W)
    
        return imgs

In [ ]:
# ======== Loss Funtion ========
class WeightedR2Loss(nn.Module):
    def __init__(self, weights=None, eps=1e-8):
        super().__init__()
        if weights is None:
            weights = torch.tensor([0.1, 0.1, 0.1, 0.2, 0.5])
        self.register_buffer('weights', weights)
        self.eps = eps

    def forward(self, y_pred, y_true):
        """
        y_pred: (B, 3)
        y_true: (B, 5)
        """
        DG = y_pred[:,0]
        GDM = y_pred[:,1]
        DT = y_pred[:,2]
        DD = DT - GDM
        DC = GDM - DG
        y_hat = torch.stack([DG, DD, DC, GDM, DT], dim=1)
        
        y_mean = torch.mean(y_true, dim=0, keepdim=True)
        ss_res = torch.sum((y_true - y_hat) ** 2, dim=0)
        ss_tot = torch.sum((y_true - y_mean) ** 2, dim=0)
        r2 = 1 - ss_res / (ss_tot + self.eps)

        weighted_r2 = torch.sum(self.weights * r2)
        loss = 1 - weighted_r2

        return loss

In [ ]:
# ======== Model ========
class MultiRegressionModel(pl.LightningModule):
    def __init__(
        self,
        model_name="efficientnet_b2",
        pretrained=False,
        lr=5e-3,
        output_dim=3,
        hidden_dim=1536,
        open_last_n_blocks=1,
        train_bn=False,
    ):
        super().__init__()
        self.save_hyperparameters()
        
        self.backbone = timm.create_model(
            model_name, 
            pretrained=pretrained, 
            num_classes=0, 
            global_pool="avg"
        )
        feat_dim = getattr(self.backbone, "num_features", None)
        if feat_dim is None:
            x_dummy = torch.zeros(1, 3, 1000, 1000)
            with torch.no_grad():
                feat_dim = self.backbone(x_dummy).shape[-1]
        self.head = nn.Sequential(
            nn.Linear(feat_dim * 2, hidden_dim),
            nn.ReLU(inplace=True),
            nn.Linear(hidden_dim, output_dim),
        )

        self.criterion = WeightedR2Loss()
        self.val_outputs = []
        self._freeze_backbone(open_last_n_blocks=open_last_n_blocks, train_bn=train_bn)

    # Helpers
    def _set_requires_grad(self, module, flag: bool):
        for p in module.parameters():
            p.requires_grad = flag
            
    def _freeze_backbone(self, open_last_n_blocks=1, train_bn=False):
        self._set_requires_grad(self.backbone, False)

        if hasattr(self.backbone, "blocks"):
            blocks = self.backbone.blocks
            n = len(blocks)
            assert open_last_n_blocks >= 1 and open_last_n_blocks <= n, \
                f"Expect open_last_n_blocks between [1, {n}], get {open_last_n_blocks}"
            for blk in blocks[n - open_last_n_blocks:]:
                self._set_requires_grad(blk, True)

        if not train_bn:
            for m in self.backbone.modules():
                if isinstance(m, (nn.BatchNorm2d, nn.SyncBatchNorm)):
                    if all(not p.requires_grad for p in m.parameters()):
                        m.eval()

        self._set_requires_grad(self.head, True)


    def forward(self, x_imgs):
        if x_imgs.ndim == 5:
            B, T, C, H, W = x_imgs.shape
            assert T == 2, f"Expect 2 pics, get T={T}"
            x = x_imgs.view(B * T, C, H, W)
            feats = self.backbone(x)
            F = feats.shape[-1]
            feats = feats.view(B, T, F)
            fused = torch.cat([feats[:, 0, :], feats[:, 1, :]], dim=1)
        elif x_imgs.ndim == 4:
            T, C, H, W = x_imgs.shape
            assert T == 2, f"Expect 2 pics, get T={T}"
            feats = self.backbone(x_imgs)
            fused = torch.cat([feats[0], feats[1]], dim=0).unsqueeze(0)
        else:
            raise ValueError("SHAPE should be train/val-(B, 2, C, H, W) or single-sample val-(2, C, H, W)")

        y_hat = self.head(fused)
        return y_hat

    def training_step(self, batch, batch_idx):
        x_img, y = batch
        y_hat = self(x_img)
        loss = self.criterion(y_hat, y)
        self.log("train_loss", loss, on_step=False, on_epoch=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x_img, y = batch
        y_hat = self(x_img)
        loss = self.criterion(y_hat, y)
        self.val_outputs.append((y_hat.detach().cpu(), y.detach().cpu()))
        self.log("val_loss", loss, on_step=False, on_epoch=True)
        return loss

    def on_validation_epoch_end(self):
        y_hats = torch.cat([x[0] for x in self.val_outputs], dim=0)
        y_trues = torch.cat([x[1] for x in self.val_outputs], dim=0)
        self.val_outputs.clear()

        DG = y_hats[:, 0]
        GDM = y_hats[:, 1]
        DT = y_hats[:, 2]
        DD = DT - GDM
        DC = GDM - DG
        y_hat_full = torch.stack([DG, DD, DC, GDM, DT], dim=1)

        y_mean = torch.mean(y_trues, dim=0, keepdim=True)
        ss_res = torch.sum((y_trues - y_hat_full) ** 2, dim=0)
        ss_tot = torch.sum((y_trues - y_mean) ** 2, dim=0)
        r2 = 1 - ss_res / (ss_tot + 1e-8)

        weights = self.criterion.weights.cpu()
        weighted_r2 = torch.sum(weights * r2)

        for i, name in enumerate(["DG", "DD", "DC", "GDM", "DT"]):
            self.log(f"val_r2_{name}", r2[i], prog_bar=True, on_epoch=True)
        self.log("val_weighted_r2", weighted_r2, prog_bar=True, on_epoch=True)

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=self.hparams.lr)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer,
            mode="min",
            factor=0.5,
            patience=2,
            threshold=0.001,
            min_lr=1e-7,
            verbose=False,
        )
        return {
            "optimizer": optimizer,
            "lr_scheduler": {
                "scheduler": scheduler,
                "monitor": "val_loss",
                "interval": "epoch",
                "frequency": 1,
            },
        }

In [ ]:
# ======== Helpers ========
def tta_inference(model, images):
    preds = model(images)
    preds_lr = model(torch.flip(images, dims=[-1]))
    preds_ud = model(torch.flip(images, dims=[-2]))
    preds_lrud = model(torch.flip(images, dims=[-1, -2]))
    preds_mean = 0.4 * preds + 0.3 * preds_lr + 0.2 * preds_ud + 0.1 * preds_lrud
    return preds_mean

In [ ]:
# ======== DataLoader ========
# Transform
img_size = 1000
infer_transform = T.Compose([
    T.Resize((img_size, img_size)),
    T.ToTensor(),
    T.Normalize([0.485, 0.456, 0.406],
                [0.229, 0.224, 0.225])
])

# DataLoader
dataset = InferenceDataset(image_paths_df, per_half_transform=infer_transform)
dataloader = DataLoader(dataset, batch_size=8, shuffle=False, num_workers=4)

In [ ]:
# ======== Infer ========
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
results_dict = {}

for fold in range(3):
    best_ckpt = f"/kaggle/input/csiro-naive-cnn-training/lightning_logs/version_{fold}/checkpoints/best_model_fold{fold}.ckpt"
    model = MultiRegressionModel.load_from_checkpoint(
        best_ckpt,
        model_name="efficientnet_b2",
        pretrained=False,
        lr=1e-4,
        output_dim=3,
        hidden_dim=1536,
        open_last_n_blocks=1,
        train_bn=False,
        map_location=device,
    )
    model.eval().to(device)

    results = []
    with torch.no_grad():
        use_amp = (device.type == "cuda")
        amp_dtype = torch.float16
        for images in dataloader:
            images = images.to(device, non_blocking=True)
            with torch.autocast("cuda", dtype=amp_dtype, enabled=use_amp):
                preds = tta_inference(model, images)
            results.append(preds.cpu().numpy())

    results_dict[fold] = np.concatenate(results, axis=0)

In [ ]:
# ======== Construct Submission ========
result_df = pd.DataFrame(np.mean([results_dict[fold] for fold in range(3)], axis=0), columns=["Dry_Green_g", "GDM_g", "Dry_Total_g"])
result_df["Dry_Dead_g"] = (result_df["Dry_Total_g"] - result_df["GDM_g"]).clip(lower=0)
result_df["Dry_Clover_g"] = (result_df["GDM_g"] - result_df["Dry_Green_g"]).clip(lower=0)
result_df['sample_id'] = image_paths_df['id']
result_df = pd.melt(result_df, id_vars='sample_id', value_vars=["Dry_Green_g", "Dry_Dead_g", "Dry_Clover_g", "GDM_g", "Dry_Total_g"], value_name='target')
result_df['sample_id'] = result_df['sample_id'] + '__' + result_df['variable']
result_df['target'] = result_df['target']
result_df[['sample_id', 'target']].to_csv('submission.csv', index=False)